# Related Media Graph
* We make an graph of items, where each item is a vertex
* Two vertices are connected if one item is a related to the other, where valid relations include sequels, prequels, etc.
* We then store the adjacency list

In [ ]:
import os

import pandas as pd
from tqdm import tqdm

In [ ]:
os.chdir("../../data/processed_data")

In [ ]:
media = ""

In [ ]:
medium = pd.read_csv(f"{media}.csv")

In [ ]:
def get_related_series(data, media_id):
    df = pd.DataFrame({"source": [], "target": [], "relationship": []}, dtype=int)
    related = eval(data)
    if not related:
        return df

    related_series = set()
    for entry in related:
        df = pd.concat(
            [
                df,
                pd.DataFrame.from_records(
                    [
                        {
                            "source": media_id,
                            "target": entry[f"{media}_id"],
                            "relationship": entry["relation"],
                        }
                    ]
                ),
            ],
            ignore_index=True,
        )
    return df

In [ ]:
all_related_series = []
for i in tqdm(range(len(medium))):
    media_id = medium.iloc[i][f"{media}_id"]
    related_series = get_related_series(medium.iloc[i][f"related_{media}"], media_id)
    all_related_series.append(related_series)
related_series_df = pd.concat(all_related_series, ignore_index=True)
related_series_df[["source", "target"]] = related_series_df[
    ["source", "target"]
].astype(int)

In [ ]:
media_to_uid = pd.read_csv(f"{media}_to_uid.csv")
media_to_uid = media_to_uid.set_index(f"{media}id").to_dict()["uid"]
related_series_df = related_series_df.loc[
    lambda x: x.source.isin(list(media_to_uid)) & x.target.isin(list(media_to_uid))
]
related_series_df["source"].replace(media_to_uid, inplace=True)
related_series_df["target"].replace(media_to_uid, inplace=True)

In [ ]:
related_series_df.to_csv(f"{media}_{media}_related_series.csv", index=False)